In [24]:
import os
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
#mpl.rc('axes', labelsize=14)
#mpl.rc('xtick', labelsize=12)
#mpl.rc('ytick', labelsize=12)

import tensorflow as tf
from tensorflow import keras

# Data API

## Dataset in ram:

In [25]:
X = tf.range(10) # 10 items 0:10
dataset = tf.data.Dataset.from_tensor_slices(X) # Equivalent: tf.data.Dataset.range(10)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [26]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


## Chaining Transformations

In [27]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [28]:
# Transformació a cada item
dataset = dataset.map(lambda x: x * 2)
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [29]:
# Transformació al dataset en general
dataset = dataset.apply(tf.data.experimental.unbatch()) # Crea batches individuals (en comptes de 7 en 7 com abans)

In [37]:
# Filtrar valors:
dataset = dataset.filter(lambda x: x < 10)
for item in dataset.take(7): # mira 7 primers items
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


## Shuffling

In [45]:
# Si cap en memòria:
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size = 5, seed = 42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


In [47]:
# Si no cap:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state = 42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state = 42)
scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [58]:
# Split the dataset:
def save_to_multiple_csv_files(data, name_prefix, header = None, n_parts = 10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok = True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding = "utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [62]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts = 20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts = 10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts = 10)

In [63]:
pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


In [64]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end = "")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


In [65]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed = 42)
for filepath in filepath_dataset.take(5):
    print(filepath)

tf.Tensor(b'datasets\\housing\\my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_00.csv', shape=(), dtype=string)


In [66]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length = n_readers)
for line in dataset.take(5):
    print(line.numpy())

b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418'
b'2.4792,24.0,3.4547038327526134,1.1341463414634145,2251.0,3.921602787456446,34.18,-118.38,2.0'
b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67'
b'2.1856,41.0,3.7189873417721517,1.0658227848101265,803.0,2.0329113924050635,32.76,-117.12,1.205'
b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215'


In [68]:
# Preprocesant
n_inputs = 8

@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype = tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults = defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

preprocess(b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.36618188, -0.99870497,  0.00781878, -0.00675364, -0.06140145,
         0.0072037 , -0.9446553 ,  0.9367464 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.418], dtype=float32)>)

In [69]:
# Ajuntat la lectura + preprocessament:
def csv_reader_dataset(filepaths, repeat = 1, n_readers = 5, n_read_threads = None, 
                       shuffle_buffer_size = 1000, n_parse_threads = 5, batch_size = 32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length = n_readers, num_parallel_calls = n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls = n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [72]:
tf.random.set_seed(42)

train_set = csv_reader_dataset(train_filepaths, batch_size = 3)
for X_batch, y_batch in train_set.take(2):
    print("X =", X_batch)
    print("y =", y_batch)
    print()

X = tf.Tensor(
[[ 0.5804519  -0.20762321  0.05616303 -0.15191229  0.01343246  0.00604472
   1.2525111  -1.3671792 ]
 [-0.21650054 -0.91959685 -0.37069115 -0.15282531  1.9280853   0.6030511
  -0.7338394   0.8018072 ]
 [-0.37792316 -0.2867314  -0.44674355 -0.02454283  1.0081758  -0.3701026
   0.79340184 -1.1822641 ]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[1.752]
 [1.522]
 [2.561]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[-1.0393791   0.02970133  0.0704432   0.01656396 -0.14901187  0.2554778
   0.69033587 -0.2876847 ]
 [-0.47738516  0.10880951 -0.23843908 -0.0527132   0.29999155  0.21409526
  -0.67293835  0.5919061 ]
 [ 0.6041258   0.8998913  -0.00953289  0.17155597 -0.63817006  0.05904346
  -0.8181658   0.6118972 ]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[0.792]
 [1.915]
 [2.319]], shape=(3, 1), dtype=float32)



In [73]:
train_set = csv_reader_dataset(train_filepaths, repeat = None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [74]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation = "relu", input_shape = X_train.shape[1:]),
    keras.layers.Dense(1),
])

In [76]:
model.compile(loss = "mse", optimizer = keras.optimizers.SGD(learning_rate = 1e-3))

In [78]:
batch_size = 32
model.fit(train_set, steps_per_epoch = len(X_train) // batch_size, epochs = 10, validation_data = valid_set)

Epoch 1/10
362/362 [==============================] - 2s 3ms/step - loss: 1.6319 - val_loss: 1.2452
Epoch 2/10
362/362 [==============================] - 0s 1ms/step - loss: 0.7099 - val_loss: 0.6582
Epoch 3/10
362/362 [==============================] - 0s 994us/step - loss: 0.6340 - val_loss: 0.5992
Epoch 4/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5979 - val_loss: 0.5675
Epoch 5/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5737 - val_loss: 0.5361
Epoch 6/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5467 - val_loss: 0.5174
Epoch 7/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5311 - val_loss: 0.5139
Epoch 8/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5118 - val_loss: 0.4802
Epoch 9/10
362/362 [==============================] - 0s 1ms/step - loss: 0.4996 - val_loss: 0.4679
Epoch 10/10
362/362 [==============================] - 0s 938us/step - loss: 0.4865 - val_loss: 0.

In [81]:
model.evaluate(test_set, steps = len(X_test) // batch_size)

161/161 [==============================] - 0s 1ms/step - loss: 0.4775


0.4774568974971771

In [90]:
# Prediccions
new_set = test_set.map(lambda X, y: X) # tambe es pot pasar test_set, keras ignoraria labels
X_new = X_test
model.predict(new_set, steps = len(X_new) // batch_size)

array([[3.0935555],
       [2.493179 ],
       [3.7334828],
       ...,
       [2.2254663],
       [3.3213046],
       [4.8779726]], dtype=float32)

In [93]:
optimizer = keras.optimizers.Nadam(learning_rate = 0.01)
loss_fn = keras.losses.mean_squared_error

n_epochs = 5
batch_size = 32
n_steps_per_epoch = len(X_train) // batch_size
total_steps = n_epochs * n_steps_per_epoch
global_step = 0
for X_batch, y_batch in train_set.take(total_steps):
    global_step += 1
    print("\rGlobal step {} / {}".format(global_step, total_steps), end = "")
    with tf.GradientTape() as tape:
        y_pred = model(X_batch)
        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

Global step 1810 / 1810

In [94]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# Capa de Preprocessament desde tf

In [103]:
means = np.mean(X_train, axis = 0, keepdims = True)
stds = np.std(X_train, axis = 0, keepdims = True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([
    keras.layers.Lambda(lambda inputs: (inputs - means) / stds + eps)
])

Si es vol optar per no tenir mitjanes voltant porai:

In [107]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis = 0, keepdims = True)
        self.stds_ = np.std(data_sample, axis = 0, keepdims = True)
    def call (self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

std_layer = Standardization()
std_layer.adapt(X_train)

# Tensorflow Datasets (TFDS)

In [118]:
import tensorflow_datasets as tfds

datasets = tfds.load(name = "mnist")
mnist_train, mnist_test = datasets["train"], datasets["test"]

In [136]:
mnist_train = mnist_train.shuffle(10000).batch(1).prefetch(1)